In [1]:
#Setup Libs
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
def get_info (filename):
    df = pd.read_parquet(filename)

    #Get the Number of Columns
    print(f"Number of columns : {len(df.columns.to_list())}")

    #Get the Duration
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    #Get The Standard divation
    print(f"standard deviation of the trips duration in January : {df['duration'].std()}")

    Filtered_Data = df[(df['duration'] <= 60) & (df['duration'] >= 1)]
    print(f"fraction of the records left after you dropped the outliers : {(Filtered_Data.shape[0]/df.shape[0])*100} %")


In [3]:
get_info('yellow_tripdata_2022-01.parquet')

Number of columns : 19
standard deviation of the trips duration in January : 46.44530513776499
fraction of the records left after you dropped the outliers : 98.27547930522405 %


In [4]:
def read_Dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    df = df[(df['duration'] <= 60) & (df['duration'] >= 1)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_Dataframe('yellow_tripdata_2022-01.parquet')
df_val = read_Dataframe('yellow_tripdata_2022-02.parquet')

In [16]:
def one_hot_encoding(dataframe):
    # Select the pickup and dropoff location ID columns
    selected_columns = ['PULocationID', 'DOLocationID']
    df_selected = dataframe[selected_columns]

    # Convert the DataFrame into a list of dictionaries
    dict_list = df_selected.to_dict(orient='records')

    # Fit a dictionary vectorizer and transform the data
    vectorizer = DictVectorizer(sparse=False)
    feature_matrix = vectorizer.fit_transform(dict_list)

    # Get the dimensionality of the feature matrix
    num_columns = feature_matrix.shape[1]

    print("Dimensionality of the feature matrix:", num_columns)
        
    return feature_matrix

In [17]:
X_train = one_hot_encoding(df_train)
X_val = one_hot_encoding(df_val)

Dimensionality of the feature matrix: 515
Dimensionality of the feature matrix: 514


In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

MemoryError: Unable to allocate 9.29 GiB for an array with shape (2421440, 515) and data type float64